# Setup

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Sample - Superstore.xls')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
pre_start = '2016-01-01'
pre_end = '2016-12-31'
post_start = '2017-01-01'
post_end = '2017-12-31'

# Ex 1 - Sales

In [4]:
pre = df[(df['Order Date'] <= pre_end) & (df['Order Date'] >= pre_start)]
post = df[(df['Order Date'] <= post_end) & (df['Order Date'] >= post_start)]

pre_sales = pre.Sales.sum()
post_sales = post.Sales.sum()
# Calculate Chg, %Chg
chg = post_sales - pre_sales
pct_chg = chg / pre_sales

print(f'''
Pre:   ${pre_sales:,.0f}
Post:  ${post_sales:,.0f}

Chg:  ${chg:+,.0f}
%Chg:  {pct_chg:+.1%}
''')


Pre:   $609,206
Post:  $733,215

Chg:  $+124,010
%Chg:  +20.4%



In [5]:
df[['Region', 'Sales']].head()

,Region,Sales
0,South,261.9600
1,South,731.9400
2,West,14.6200
3,South,957.5775
4,South,22.3680


## Calculate Sales for Each Region

- how can you subset the pre and post dataframes to achieve this?
- Is there any other programmatic solution you can think of?
- (Hint: consider adding a pre_post_label column to the dataframe)

# Ex 2 - Sales by Region

## Solution 1: Use a loop to look at unique regions

In [6]:
regions = df.Region.unique()
for region in regions:
    pre_sales = pre[pre.Region == region].Sales.sum()
    post_sales = post[post.Region == region].Sales.sum()
    chg = post_sales - pre_sales
    pct_chg = chg / pre_sales

    print(f'''
    --- Region: {region}
    Pre:   ${pre_sales:,.0f}
    Post:  ${post_sales:,.0f}

    Chg:  ${chg:+,.0f}
    %Chg:  {pct_chg:+.1%}
    ''')


    --- Region: South
    Pre:   $93,610
    Post:  $122,906

    Chg:  $+29,296
    %Chg:  +31.3%
    

    --- Region: West
    Pre:   $187,480
    Post:  $250,128

    Chg:  $+62,648
    %Chg:  +33.4%
    

    --- Region: Central
    Pre:   $147,429
    Post:  $147,098

    Chg:  $-331
    %Chg:  -0.2%
    

    --- Region: East
    Pre:   $180,686
    Post:  $213,083

    Chg:  $+32,397
    %Chg:  +17.9%
    


## Solution 2: Use a pre_post_label column, aggregation, and reshaping

In [7]:
df = pd.read_excel('Sample - Superstore.xls')
df = df[df['Order Date'] >= pre_start]
df = df[df['Order Date'] <= post_end]
df['pre_post_label'] = np.where(df['Order Date'] <= pre_end, 'pre', 'post')
df[['Order Date', 'Region', 'Sales', 'pre_post_label']].head(15)

,Order Date,Region,Sales,pre_post_label
0,2016-11-08,South,261.960,pre
1,2016-11-08,South,731.940,pre
2,2016-06-12,West,14.620,pre
12,2017-04-15,South,15.552,post
13,2016-12-05,West,407.976,pre
21,2016-12-09,Central,19.460,pre
22,2016-12-09,Central,60.340,pre
23,2017-07-16,East,71.372,post
25,2016-01-16,West,11.648,pre
26,2016-01-16,West,90.570,pre


In [8]:
# df.pivot_table('Sales', 'Region', 'pre_post_label', aggfunc=np.sum)

In [9]:
df.groupby(['Region', 'pre_post_label'], as_index=False).Sales.sum()

,Region,pre_post_label,Sales
0,Central,post,147098.1282
1,Central,pre,147429.3760
2,East,post,213082.9040
3,East,pre,180685.8220
4,South,post,122905.8575
5,South,pre,93610.2235
6,West,post,250128.3655
7,West,pre,187480.1765


In [10]:
t = df.groupby(['Region', 'pre_post_label']).Sales.sum().unstack()
t['chg'] = t.post - t.pre
t['pct_chg'] = t.chg / t.pre
t.style.format('${:,.0f}', subset=['pre', 'post']).format('${:+,.0f}', subset='chg').format('{:+.1%}', subset='pct_chg')

pre_post_label,post,pre,chg,pct_chg
Region,,,,
Central,"$147,098","$147,429",$-331,-0.2%
East,"$213,083","$180,686","$+32,397",+17.9%
South,"$122,906","$93,610","$+29,296",+31.3%
West,"$250,128","$187,480","$+62,648",+33.4%
